In [17]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

In [18]:
path = os.listdir('C:/Users/garvm/OneDrive/Desktop/genderdatas/train')
classes = {'Female' : 0 , 'Male' : 1}

In [19]:
X = []
Y = []
for cls in classes:
    subpath = 'C:/Users/garvm/OneDrive/Desktop/genderdatas/train/'+cls
    for j in os.listdir(subpath):
        img_path = os.path.join(subpath, j)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
        if img is not None:
            img = cv2.resize(img, (100, 100))
            img = img.reshape(100, 100, 1)
            X.append(img)
            Y.append(classes[cls])


In [20]:
X = np.array(X)
Y = np.array(Y)
print("Class distribution:", pd.Series(Y).value_counts())


Class distribution: 0    3000
1    3000
Name: count, dtype: int64


In [21]:
X = X.astype('float32') / 255.0

In [22]:
lb = LabelBinarizer()
Y = lb.fit_transform(Y)
Y = to_categorical(Y)


In [23]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state=10, test_size=0.20, stratify=Y)


In [24]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\garvm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(xtrain, ytrain,epochs=30,batch_size=32,validation_data=(xtest, ytest),callbacks=[early_stopping])

test_loss, test_accuracy = model.evaluate(xtest, ytest)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")



Epoch 1/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 47s 278ms/step - accuracy: 0.7376 - loss: 1.0534 - val_accuracy: 0.6550 - val_loss: 0.5903
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 79s 261ms/step - accuracy: 0.9102 - loss: 0.2374 - val_accuracy: 0.8092 - val_loss: 0.4122
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 42s 281ms/step - accuracy: 0.9542 - loss: 0.1393 - val_accuracy: 0.9258 - val_loss: 0.1808
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 42s 279ms/step - accuracy: 0.9682 - loss: 0.0816 - val_accuracy: 0.9317 - val_loss: 0.1990
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 269ms/step - accuracy: 0.9734 - loss: 0.0768 - val_accuracy: 0.9408 - val_loss: 0.1895
Epoch 6/30
 28/150 ━━━━━━━━━━━━━━━━━━━━ 29s 242ms/step - accuracy: 0.9723 - loss: 0.0629

In [ ]:
y_pred = model.predict(xtest)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(ytest, axis=1)

In [ ]:
print("\nClassification Report:\n", classification_report(y_true, y_pred_classes))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred_classes))

In [ ]:
def predict_gender():
    testimgpath = "C:\\Users\\garvm\\OneDrive\\Desktop\\Pendrive data\\trip\\Himachal\\IMG20170603103433.jpg"
    # Read the image
    img = cv2.imread(testimgpath, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Image not found or invalid")
        return

    # Resize the image
    img = cv2.resize(img, (100, 100))

    # Reshape for model input
    img = img.reshape(1, 100, 100, 1) / 255.0

    # Make the prediction using your model
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)

    if predicted_class == 0:
        return 'Female'
    else:
        return 'Male'
print("Predicted gender:", predict_gender())        